In [ ]:
!apt-get update
!apt-get install ffmpeg
!pip install yt-dlp

In [12]:
import requests
##---------------------------------------Previous 100 URL**


# Base URL for NRK's API
base_url = "https://psapi.nrk.no"

# Direkte URL for "Debatten" serien
series_url = f"{base_url}/tv/catalog/series/debatten"

# Antall episoder vi ønsker å hente
target_episodes = 100
episodes_fetched = 0
page = 1

try:
    all_episodes = []

    while episodes_fetched < target_episodes:
        response = requests.get(series_url, params={"page": page})
        response.raise_for_status()
        series_data = response.json()

        if '_embedded' in series_data and 'instalments' in series_data['_embedded']:
            instalment_data = series_data['_embedded']['instalments']
            if '_embedded' in instalment_data and 'instalments' in instalment_data['_embedded']:
                episodes = instalment_data['_embedded']['instalments']
                all_episodes.extend(episodes)
                episodes_fetched += len(episodes)

                # Sjekk om det er flere sider
                if '_links' in instalment_data and 'next' in instalment_data['_links']:
                    page += 1
                else:
                    break  # Ingen flere sider
            else:
                break  # Ingen flere episoder funnet
        else:
            break  # Ingen flere episoder funnet

    print(f"Episoder av Debatten (totalt {len(all_episodes)}):")
    for episode in all_episodes[:target_episodes]:
        episode_id = episode['prfId']
        episode_title = episode['titles']['title']
        episode_date = episode['usageRights']['from']['displayValue']
        episode_url = f'https://tv.nrk.no/se?v={episode_id}'
        print(f"Dato: {episode_date}")
        print(f"Tittel: {episode_title}")
        print(f"URL: {episode_url}")
        print("---")

except requests.RequestException as e:
    print(f"En feil oppstod ved henting av data: {e}")

Episoder av Debatten (totalt 100):
Dato: 22. oktober 2024 kl. 21:15
Tittel: 22. oktober
URL: https://tv.nrk.no/se?v=NNFA51102224
---
Dato: 17. oktober 2024 kl. 21:15
Tittel: Torsdag · Slik kan du spare på maten
URL: https://tv.nrk.no/se?v=NNFA51101724
---
Dato: 15. oktober 2024 kl. 21:15
Tittel: 15. oktober · Skoler legges ned, bygder dør
URL: https://tv.nrk.no/se?v=NNFA51101524
---
Dato: 10. oktober 2024 kl. 21:15
Tittel: 10. oktober · Bløffebudsjett, mener opposisjonen
URL: https://tv.nrk.no/se?v=NNFA51101024
---
Dato: 8. oktober 2024 kl. 21:15
Tittel: 8. oktober · Tidenes budsjett, men misnøyen vokser
URL: https://tv.nrk.no/se?v=NNFA51100824
---
Dato: 3. oktober 2024 kl. 21:15
Tittel: 3. oktober · Hvorfor bomber de hverandre?
URL: https://tv.nrk.no/se?v=NNFA51100324
---
Dato: 1. oktober 2024 kl. 21:16
Tittel: 1. oktober · De sa vi skulle få bedre råd
URL: https://tv.nrk.no/se?v=NNFA51100124
---
Dato: 26. september 2024 kl. 21:15
Tittel: 26. september · Ber Abid Raja slutte å spille 

In [14]:
!pip install yt-dlp os

import yt_dlp
import os

##---------------------------------------Test for subtitles download**


def main():
    url = "https://tv.nrk.no/serie/debatten/sesong/202410/episode/NNFA51101724"

    download_path = os.path.join(os.getcwd(), "Debatten_subtitles")
    os.makedirs(download_path, exist_ok=True)

    ydl_opts = {
        'writesubtitles': True,
        'subtitleslangs': ['nb-ttv'],  # Norsk bokmål TV-teksting
        'skip_download': True,  # Hopp over nedlasting av video
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),
        'quiet': False,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegSubtitlesConvertor',
            'format': 'srt',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"Subtitles downloaded to -> : {download_path}")
    except Exception as e:
        print(f"En feil oppstod: {e}")

if __name__ == "__main__":
    main()

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
[NRKTV] Extracting URL: https://tv.nrk.no/serie/debatten/sesong/202410/episode/NNFA51101724
[NRK] Extracting URL: nrk:NNFA51101724
[NRK] NNFA51101724: Downloading manifest JSON
[NRK] NNFA51101724: Downloading m3u8 information
[NRK] NNFA51101724: Downloading metadata JSON
[NRK] NNFA51101724: Downloading programs JSON
[info] NNFA51101724: Downloading subtitles: nb-ttv
[info] NNFA51101724: Downloading 1 format(s): 2344+aud2-Unknown
[info] Writing video subtitles to: /content/Debatten_subtitles/Debatten - Torsdag · Slik kan du spare på maten.nb-ttv.vtt
[download] Destination: /content/Debatten_subtitles/Debatten - Torsdag · Slik kan du spare på maten.nb-ttv.vtt
[download] 100% of   38.96KiB in 00:00:00 at 279.18KiB/s 
Subtitles downloaded to -> : /content/Debatten_subtitles


In [11]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import yt_dlp

##-----------------------------------Implementation with the API/ Scaling

# Base URL for NRK's API
base_url = "https://psapi.nrk.no"
series_url = f"{base_url}/tv/catalog/series/debatten"

def get_episode_urls():
    """
    Fetch episode URLs and titles from the NRK API for 'Debatten'.

    Returns:
    list: A list of tuples containing (episode_url, episode_title).
    """
    try:
        response = requests.get(series_url)
        response.raise_for_status()
        series_data = response.json()

        if '_embedded' in series_data and 'instalments' in series_data['_embedded']:
            instalment_data = series_data['_embedded']['instalments']
            if '_embedded' in instalment_data and 'instalments' in instalment_data['_embedded']:
                episodes = instalment_data['_embedded']['instalments']
                return [(f'https://tv.nrk.no/se?v={episode["prfId"]}', episode['titles']['title']) for episode in episodes]
    except requests.RequestException as e:
        print(f"An error occurred while fetching episodes: {e}")
    return []

def download_subtitles(url, title, download_path):
    """
    Download subtitles for a given episode using yt-dlp.

    Args:
    url (str): The URL of the episode.
    title (str): The title of the episode.
    download_path (str): The path where subtitles will be saved.

    Returns:
    str: A status message indicating success or failure.
    """
    ydl_opts = {
        'writesubtitles': True,
        'subtitleslangs': ['nb-ttv'],  # Norwegian Bokmål TV subtitles
        'skip_download': True,  # Skip video download
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),
        'quiet': True,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegSubtitlesConvertor',
            'format': 'srt',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return f"Subtitles downloaded for: {title}"
    except Exception as e:
        return f"Error downloading subtitles for {title}: {e}"

def combine_subtitles(download_path, output_file):
    """
    Combine individual subtitle files into a single text file.

    Args:
    download_path (str): The path where individual subtitle files are stored.
    output_file (str): The path for the combined output file.
    """
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for filename in os.listdir(download_path):
            if filename.endswith('.srt'):
                episode_title = os.path.splitext(filename)[0]
                outfile.write(f"\n\n{'='*50}\nEPISODE: {episode_title}\n{'='*50}\n\n")

                with open(os.path.join(download_path, filename), 'r', encoding='utf-8') as infile:
                    subtitle_text = infile.read()
                    outfile.write(subtitle_text + "\n")

def main():
    """
    Main function to coordinate subtitle downloading and combination.
    """
    download_path = os.path.join(os.getcwd(), "Debatten_Undertekster")
    os.makedirs(download_path, exist_ok=True)

    episode_info = get_episode_urls()
    print(f"Found {len(episode_info)} episodes.")

    # Use ThreadPoolExecutor to download subtitles concurrently
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(download_subtitles, url, title, download_path): title for url, title in episode_info}
        for future in as_completed(future_to_url):
            print(future.result())


if __name__ == "__main__":
    main()




Found 10 episodes.


ERROR: [NRK] NNFA51102224: NRK said: Kommer i morgen kl. 21:16


Error downloading subtitles for 22. oktober: ERROR: [NRK] NNFA51102224: NRK said: Kommer i morgen kl. 21:16
Subtitles downloaded for: Torsdag · Slik kan du spare på maten
Subtitles downloaded for: 8. oktober · Tidenes budsjett, men misnøyen vokser
Subtitles downloaded for: 3. oktober · Hvorfor bomber de hverandre?
Subtitles downloaded for: 10. oktober · Bløffebudsjett, mener opposisjonen
Subtitles downloaded for: 15. oktober · Skoler legges ned, bygder dør
Subtitles downloaded for: 1. oktober · De sa vi skulle få bedre råd
Subtitles downloaded for: 26. september · Ber Abid Raja slutte å spille på fremmedfrykt
Subtitles downloaded for: 24. september · Sykelønnsforslag provoserer
Subtitles downloaded for: 19. september · Ny bok: Støre forsøkt kastet
